### Asynchronous SGD updates using Distributed Cache Redis

1. Create spot instances
2. Mount EFS (Not required for our project)
3. Start redis server in all the instances and create a cluster
4. Pull the code from github repo (https://github.com/SrujithPoondla/vanilla-hogwild.git)
5. If need to divide the dataset between the nodes run the specific cell
6. Run the scripts to start training
7. After training ends close the instances

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from aws_setup import *
from argparse import ArgumentParser
import boto3
import os 
sys.argv = ['foo']

#### Define parameters

In [3]:
parser = ArgumentParser(description='Asynchronous SGD updates using Redis')
parser.add_argument('--n-nodes', type=int, default=1, metavar='N',
                    help='how many aws instances to start')
parser.add_argument('--is-redis', action='store_true', default = True,
                    help="Choose whether the model to be trained using redis or not."
                    "If not using Redis, model will be trained on single process")


_StoreTrueAction(option_strings=['--is-redis'], dest='is_redis', nargs=0, const=True, default=True, type=None, choices=None, help='Choose whether the model to be trained using redis or not.If not using Redis, model will be trained on single process', metavar=None)

In [4]:
args = parser.parse_args()
vpc_name='vpc-1b056b60'
args.n_nodes = 8
n_instances = args.n_nodes
instance_type = 'm5.2xlarge'
ami_sr = 'ami-f679ff89'
a_zone = 'us-east-1a'

if args.n_nodes is 2 and args.is_redis:
    print('Cant create a cluster with 2 redis nodes. Chose either a 3 node cluster or single instance')

#### Get Existing VPC by tag name

In [5]:
vpc = get_vpc(vpc_name); vpc

ec2.Vpc(id='vpc-1b056b60')

#### Create EFS (if you haven't already)

In [6]:
# efs_tag = f'{vpc_name}-efs'

In [7]:
# efs = create_efs(efs_tag, vpc, performance_mode='maxIO')

#### Request Spot instance

In [8]:
instance_name = f'{vpc_name}-instance'
# Recommend a high compute instance as we need to do multi-threaded resizing later on

In [9]:
spot_price = get_spot_prices()[instance_type]
bid_price = "%.4f" % (float(spot_price)*3)
print(f'Spot price: {spot_price}, Bid price: {bid_price}')

Spot price: 0.126400, Bid price: 0.3792


In [10]:
launch_specs = LaunchSpecs(vpc, instance_type=instance_type, ami= ami_sr,availability_zone=a_zone).build()

In [11]:
# launch_specs['BlockDeviceMappings'][0]['Ebs']['VolumeSize'] = 1000

In [12]:
launch_specs

{'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
   'Ebs': {'DeleteOnTermination': True,
    'VolumeSize': 20,
    'VolumeType': 'gp2'}}],
 'ImageId': 'ami-f679ff89',
 'InstanceType': 'm5.2xlarge',
 'KeyName': 'aws-key-spot-instance',
 'NetworkInterfaces': [{'AssociatePublicIpAddress': True,
   'DeviceIndex': 0,
   'Groups': ['sg-2624da6f'],
   'SubnetId': 'subnet-10d9d04d'}]}

In [13]:
ec2 = boto3.resource('ec2')
filters = [
    {
        'Name': 'instance-state-name',
        'Values': ['running']
    }
]
# filter the instances based on filters() above
ec2_instances = list(ec2.instances.filter(Filters=filters))
instances = []
for instance in ec2_instances:
    instances.append(instance)
instances_to_request = n_instances-len(instances)
instances = create_multiple_spot_instance(instance_name, launch_specs,instance_count=args.n_nodes, spot_price=bid_price)
print(instances)

Keypair exists
Waiting on spot fullfillment...
Fulfillment completed. InstanceId: i-05c5b16a3ed11e3c5
Waiting on spot fullfillment...
Fulfillment completed. InstanceId: i-024da8c6a575c63c2
Waiting on spot fullfillment...
Fulfillment completed. InstanceId: i-08751280993098d3c
Waiting on spot fullfillment...
Fulfillment completed. InstanceId: i-08c716a4a5757b0db
Waiting on spot fullfillment...
Fulfillment completed. InstanceId: i-0fc573f549033dd8d
Waiting on spot fullfillment...
Fulfillment completed. InstanceId: i-012a00a6d61d6dbdb
Waiting on spot fullfillment...
Fulfillment completed. InstanceId: i-016fe26331d549ad1
Waiting on spot fullfillment...
Fulfillment completed. InstanceId: i-002013ecb90d32339
Rebooting...
Completed. SSH:  ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@34.207.134.30
Completed. SSH:  ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@184.72.93.174
Completed. SSH:  ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@35.171.184.36
Completed. SSH:  ssh -i ~/.ssh/aws-key-spo

In [14]:
# instance = get_instance(instance_name); instance
ssh_commands = []
public_ip_list = []
private_ip_list = []
for instance in instances:
    # for each instance, append to lists
    private_ip_list.append(instance.private_ip_address)
    public_ip_list.append(instance.public_ip_address)
    ssh_commands.append(get_ssh_command(instance))
print(ssh_commands, public_ip_list, private_ip_list)

['ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@34.207.134.30', 'ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@184.72.93.174', 'ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@35.171.184.36', 'ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@52.90.119.145', 'ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@35.168.17.216', 'ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@34.238.169.132', 'ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@54.152.50.63', 'ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@35.153.138.74'] ['34.207.134.30', '184.72.93.174', '35.171.184.36', '52.90.119.145', '35.168.17.216', '34.238.169.132', '54.152.50.63', '35.153.138.74'] ['10.0.0.4', '10.0.0.9', '10.0.0.8', '10.0.0.6', '10.0.0.5', '10.0.0.13', '10.0.0.12', '10.0.0.11']


#### Terminating instances

In [15]:
# for instance in instances:
#     print(instance.terminate())

### SSH

In [16]:

def get_ssh_clients(instances):
    clients = []
    for instance in instances:
        clients.append(connect_to_instance(instance))
    print(clients)
    return clients
clients = get_ssh_clients(instances)

Connecting to SSH...
Got client
/Users/srujithpoondla/.ssh/aws-key-spot-instance.pem
Exception: timed out Retrying...
Connected!
Connecting to SSH...
Got client
/Users/srujithpoondla/.ssh/aws-key-spot-instance.pem
Connected!
Connecting to SSH...
Got client
/Users/srujithpoondla/.ssh/aws-key-spot-instance.pem
Connected!
Connecting to SSH...
Got client
/Users/srujithpoondla/.ssh/aws-key-spot-instance.pem
Connected!
Connecting to SSH...
Got client
/Users/srujithpoondla/.ssh/aws-key-spot-instance.pem
Exception: [Errno None] Unable to connect to port 22 on 35.168.17.216 Retrying...
Connected!
Connecting to SSH...
Got client
/Users/srujithpoondla/.ssh/aws-key-spot-instance.pem
Connected!
Connecting to SSH...
Got client
/Users/srujithpoondla/.ssh/aws-key-spot-instance.pem
Connected!
Connecting to SSH...
Got client
/Users/srujithpoondla/.ssh/aws-key-spot-instance.pem
Connected!
[<paramiko.client.SSHClient object at 0x1135097f0>, <paramiko.client.SSHClient object at 0x1136033c8>, <paramiko.clie

#### Mount EFS

In [17]:
# efs_addr = get_efs_address('fast-ai-efs'); efs_addr

In [18]:
# _ = run_command(client, 'mkdir ~/efs_mount')

In [19]:
# efs_mount_cmd = f'sudo mount -t nfs -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 {efs_addr}:/ ~/efs_mount'
# _ = run_command(client, efs_mount_cmd)

In [20]:
# _ = run_command(client, 'ls efs_mount') # no reformatting

## Tmux

In [21]:
def get_tmux_sess(clients):
    tsess = []
    for client in clients:
        if 'sess' not in run_command(client,'tmux ls'):
            tsess.append(TmuxSession(client, 'sess'))         
    print(tsess)
    return tsess
tsess = get_tmux_sess(clients)

[<aws_setup.TmuxSession object at 0x113611cc0>, <aws_setup.TmuxSession object at 0x113ab5630>, <aws_setup.TmuxSession object at 0x113ab59e8>, <aws_setup.TmuxSession object at 0x113aa1c50>, <aws_setup.TmuxSession object at 0x113abe4a8>, <aws_setup.TmuxSession object at 0x113abe828>, <aws_setup.TmuxSession object at 0x113ac8438>, <aws_setup.TmuxSession object at 0x113ac8080>]


#### Activate Conda Environment in all the instances and check whether we need to create a cluster or not. Then start redis using conf files.

In [22]:
for client,sess in zip(clients,tsess):
    if 'redis' not in run_command(client,'tmux ls'):
        redis = TmuxSession(client, 'redis-sess')
    run_command(client, 'cd ~/vanilla-hogwild && git stash && git pull && git checkout stable')
    if (args.n_nodes >= 3):
        redis.run_command('rm dump.rdb && rm appendonly.aof && rm nodes-6379.conf')
        print(redis.run_command('~/miniconda3/envs/largescale/bin/redis-server ~/redis-conf/redis_cluster.conf'))
    else:
        redis.run_command('rm dump.rdb && rm appendonly.aof && rm nodes-6379.conf')
        print(redis.run_command('~/miniconda3/envs/largescale/bin/redis-server ~/redis-conf/redis.conf'))
  

('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')


#### Create redis cluster

In [23]:
if args.n_nodes >= 3:
    ip_str = ''
    for ip in private_ip_list:
        ip_str = ip_str+ ip +":6379 "
    print(ip_str)
    redis = TmuxSession(clients[0],'redis-serv-sess')
    redis.run_command('cd /home/ubuntu/redis-4.0.9/src && ./redis-trib.rb create '+ ip_str)
    redis.run_command('yes')
    redis.run_command('exit')
    redis.run_command('exit')
    redis.run_command('exit')
    redis.run_command('exit')


10.0.0.4:6379 10.0.0.9:6379 10.0.0.8:6379 10.0.0.6:6379 10.0.0.5:6379 10.0.0.13:6379 10.0.0.12:6379 10.0.0.11:6379 


#### Creating Arguments String

In [24]:
#Model parameters
batch_size = 128
epochs = 3
lr = 0.01
momentum = 0.5
log_interval = 50
num_processes = 1
nnet_arch = 'LeNet'
dataset = 'cifar10'
args.is_redis = True


In [25]:
hosts = ''
for ip in private_ip_list:
    hosts = hosts+ip+','
hosts = hosts.strip(',')

In [26]:
#dataset can be 'MNIST' or 'cifar10'
#architecture can be 'LeNet' or 'ResNet'(still working on this)
#num_processes should be either 1 or 2
#batch size 128,256,512,1024,2048

arg_str = '--is-redis='+str(args.is_redis)+' --dataset='+dataset+' --nnet-arch='+nnet_arch+' --num-processes='+ str(num_processes) + ' --batch-size='+str(batch_size) +' --lr='+str(lr) + ' --hosts='+hosts +' --epochs='+str(epochs)

print(arg_str)

--is-redis=True --dataset=cifar10 --nnet-arch=LeNet --num-processes=1 --batch-size=128 --lr=0.01 --hosts=10.0.0.4,10.0.0.9,10.0.0.8,10.0.0.6,10.0.0.5,10.0.0.13,10.0.0.12,10.0.0.11 --epochs=3


#### Chose the log file name

In [27]:
if args.is_redis:
    log_file = dataset+'-'+nnet_arch+'-'+str(batch_size)+'-'+str(num_processes)+'-'+ 'redis'
else:
    log_file = dataset+'-'+nnet_arch+'-'+str(batch_size)+'-'+str(num_processes)
print('Log file name: '+log_file)

Log file name: cifar10-LeNet-128-1-redis


In [28]:
out = run_command(clients[0],'w')[0].split(',')[1].strip().split(' ')[0]
sleep(40)

In [29]:
count = 0
running = True
for sess,ip in zip(tsess,private_ip_list):
    sess.run_command('source activate largescale')
    sess.run_command('~/miniconda3/envs/largescale/bin/redis-cli -h '+str(ip) +' flushall')
    sess.run_command('python3 -u ~/vanilla-hogwild/main.py '+arg_str+ ' 2>&1 | tee '+log_file)

# sleep(100)
while(running):
    sleep(10)
    for client in clients:
        try:
            out=run_command(client,'pidof python3')[0]
#             print(out.strip('').split(' '))
            if len(out.strip(' ').split(' ')[0])>1:
#                 print(run_command(client,'pidof python3')[0].strip(' ').split(' '))
                continue
            else:
                count = count+1
        except Exception as e:
            print(e)
        if count==len(clients):
            running = False
            break
# sleep(30)
path = "/Users/srujithpoondla/lsml_results/"+str(args.n_nodes)+"/"
if not os.path.exists(path):
    os.mkdir(path)
for i,client in enumerate(clients):
    sftp = client.open_sftp()
    print(path+log_file+str(i))
    sftp.get(log_file, path+log_file+str(i))
    print('copied README back here')


/Users/srujithpoondla/lsml_results/8/cifar10-LeNet-128-1-redis0
copied README back here
/Users/srujithpoondla/lsml_results/8/cifar10-LeNet-128-1-redis1
copied README back here
/Users/srujithpoondla/lsml_results/8/cifar10-LeNet-128-1-redis2
copied README back here
/Users/srujithpoondla/lsml_results/8/cifar10-LeNet-128-1-redis3
copied README back here
/Users/srujithpoondla/lsml_results/8/cifar10-LeNet-128-1-redis4
copied README back here
/Users/srujithpoondla/lsml_results/8/cifar10-LeNet-128-1-redis5
copied README back here
/Users/srujithpoondla/lsml_results/8/cifar10-LeNet-128-1-redis6
copied README back here
/Users/srujithpoondla/lsml_results/8/cifar10-LeNet-128-1-redis7
copied README back here


#### Terminate all instances

In [30]:
for instance in instances:
    print(instance.terminate())

{'TerminatingInstances': [{'CurrentState': {'Code': 32, 'Name': 'shutting-down'}, 'InstanceId': 'i-024da8c6a575c63c2', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': 'b3e15be9-f37c-431f-b3a3-d3debcc1b407', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8', 'transfer-encoding': 'chunked', 'vary': 'Accept-Encoding', 'date': 'Sun, 06 May 2018 03:36:04 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}
{'TerminatingInstances': [{'CurrentState': {'Code': 32, 'Name': 'shutting-down'}, 'InstanceId': 'i-0fc573f549033dd8d', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': '4cbf45e9-c090-41ff-9aa0-826765ae6eae', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8', 'transfer-encoding': 'chunked', 'vary': 'Accept-Encoding', 'date': 'Sun, 06 May 2018 03:36:05 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}
{'TerminatingInstances': [{'CurrentState': {'Code': 32, 'N

In [31]:
l =[ 1,2,3]
l = l[:3]

In [32]:
l

[1, 2, 3]